## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint

`(3) 벡터저장소 로드`  
- 저장해 둔 크로마 벡터저장소를 가져오기

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 이전에 사용했던 임베딩 모델을 똑같이 설정을 해준다.
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)
# ChromaDB에서 저장한것을 가져오기 위해서
# Chroma 객체체에서 임베딩,벡터 저장소로 저장한 값들을 가져와야한다.
vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 5


## 2. LangChain LCEL

### 2.1 Prompt + LLM

In [3]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, # 좀더 창의적인 답변을 할 수 있도록 조금 설정을 해준 값
    max_tokens=100,
    )
#튜플 형태로 사용
messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

input_variables=['query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]


In [5]:
# 템플릿 입력 변수를 출력
print(prompt.input_variables)

['query']


In [6]:
# input 값을 전달하여 프롬프트를 렌더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")

print(prompt_text)

System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?


In [7]:
# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)

# 모델의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하여 회사를 이끌어왔습니다.


In [24]:
# LCEL 체인을 구성
chain = prompt | llm

# 체인을 출력
print(chain)

first=ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]) middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x15c005f70>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10979a3c0>, root_client=<openai.OpenAI object at 0x1097ac8f0>, root_async_client=<openai.AsyncOpenAI object at 0x15c005fa0>, model_name='gpt-4o-mini', temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=100)


In [25]:
# 체인의 입력 스키마를 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


/var/folders/sl/ly32t5wx3p5b36cvk238t1840000gn/T/ipykernel_18561/995298498.py:3: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pprint(chain.input_schema.schema())


In [26]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 이후 엘론 머스크가 2004년에 투자자로 참여하면서 CEO로 취임하게 되었고, 회사의 비전과 방향성을 주도하게 되었습니다. 따라서 엘론 머스크는


In [27]:
# 체인을 실행 - 옵션 2
response = chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
print(response.content)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 기여를 하였습니다. 따라서 테슬라의 창립자들 중


### 2.2 Prompt + LLM + Output Parser

`a) 문자열 파싱 - StrOutputParser`

In [28]:
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 기여를 하였습니다. 따라서 테슬라의 창립자들 중', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 27, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'length', 'logprobs': None}, id='run-d94890b2-08a1-4055-866b-dfd3dd6c1091-0', usage_metadata={'input_tokens': 27, 'output_tokens': 100, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [29]:
# StrOutputParser - 문자열 출력을 파싱
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser()

# 출력 파서를 실행
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 기여를 하였습니다. 따라서 테슬라의 창립자들 중'

In [30]:
str_chain = prompt | llm  | output_parser

query = "리비안의 설립년도는 언제인가요?"# 모델에 질문
str_response = str_chain.invoke(query)
#chain은 프롬프트와 llm만 연결이 되어있는 상태
print(str_response)



리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.


`b) JSON 출력 - JsonOutputParser`

In [31]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행 (JSON 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.") 
print(json_response)

# 출력 파서를 실행
json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": {\n    "이름": "엘론 머스크",\n    "출생연도": 1971,\n    "국적": "미국",\n    "직업": "기업가, 엔지니어, 발명가"\n  }\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 34, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None} id='run-9d3c1bc5-2047-4f46-952b-129211436b5a-0' usage_metadata={'input_tokens': 34, 'output_tokens': 64, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
{'창업자': {'이름': '엘론 머스크', '출생연도': 1971, '국적': '미국', '직업': '기업가, 엔지니어, 발명가'}}


`c) Schema 지정 - PydanticOutputParser`

In [35]:
#pip show pydantic

## pydantic_v1 버전의 스타일
    - v1버전으로 바꿀려면
        pip install "pydantic<2.0.0"

In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
#pydantic을 처리 하기 위해서 필요한 객체 BaseModel과 Filed 객체
# 24.12.12 강의에서의 버전은 v1버전을 기준으로 설명을 하기 때문에
# 나는 v2의 버전을 사용하면서 작성하는 스타일이 변경이 되어서 수정을 해야한다.
# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")
# 이름과 그 사람의 title이나 직위 2가지 속성을 관리를 하겠다는 설정
# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person) # 2가지 속성을 관리하겠다는 설정 객체를 전달
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions()) # 프롬프트 구조를 직접 전달하는 구조가 format_instructions
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")


ModuleNotFoundError: No module named 'langchain_core.pydantic_v2'

## pydantic_v2버전의 스타일

In [37]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
#pydantic을 처리 하기 위해서 필요한 객체 BaseModel과 Filed 객체
# 24.12.12 강의에서의 버전은 v1버전을 기준으로 설명을 하기 때문에
# 나는 v2의 버전을 사용하면서 작성하는 스타일이 변경이 되어서 수정을 해야한다.
# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")
    
    model_config = {
        "json_schema_extra": {
            "examples": [
                {
                    "name": "일론 머스크",
                    "title": "테슬라 CEO"
                }
            ]
        }
    }
# 이름과 그 사람의 title이나 직위 2가지 속성을 관리를 하겠다는 설정
# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person) # 2가지 속성을 관리하겠다는 설정 객체를 전달
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions()) # 프롬프트 구조를 직접 전달하는 구조가 format_instructions
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")


PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "examples": [{"name": "일론 머스크", "title": "테슬라 CEO"}], "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "title": {"description": "The title or position of the person.", "title": "Title", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatt

In [38]:
# 체인을 구성
person_chain = prompt | llm | person_parser

# 체인을 실행
response = person_chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
response

Person(name='일론 머스크', title='테슬라 CEO')

## 3. Chat Completion Methods
    - invoke -> 완성된 출력을 전달
    - stream -> 입력에 대한 응답을 실시간으로 생성되는 토큰을 단어마다 생성하여 출력
    - batch -> 입력 리스트에 대한 응답을 배치 단위로 생성하여 출력

`(1) stream`  
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [21]:
import time 

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # 기본적으로 print 함수는 출력을 할 때마다 줄바꿈을 하지만, 줄바꿈 없이 출력하려면 end=""를 사용하면 됩니다.
    # flush=True 옵션을 사용하여 출력 버퍼를 즉시 비웁니다. 데이터를 지연 없이 즉시 출력하는 데 유용합니다. 메모리를 효율적으로 출력을 한다.
    print(chunk.content, end="", flush=True)  
    # time.sleep(0.1)  # 0.1초 대기 (100ms)
    # 토큰 단위로 히턴을 한다.

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 이후 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다.

`(2) batch`  
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [22]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()
#2개의 입력에 대한 배치를 처리를 해서 각각의 응답 배치를 출력을 한다.

================================== Ai Message ==================================

테슬라의 창립자는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)입니다. 이 두 사람은 2003년에 테슬라 모터스를 설립했습니다. 이후 일론 머스크(Elon Musk)가 2004년에 투자자로 참여하면서 회사의 주요 인물 중 하나가 되었고, 이후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다.

================================== Ai Message ==================================

리비안(Rivian)의 창업자는 RJ 스칼링(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 및 친환경 차량 개발에 주력하고 있습니다. 리비안은 전기 픽업트럭인 R1T와 전기 SUV인 R1S를 출시하여 주목받고 있습니다.



## 4. Runnable
    - LCEL의 핵심 개념으로 다양한 컴포넌트를 연결하고 실행하는 인터페이스
    - 대표적으로 RunnableParallel,RunnablePassthrough,RunnnableLambda가 주로 쓰인다.

`(1) RunnableParallel`
    - 병렬 실행을 담당을 하고 있다.
    - 여러 가지를 인터페이스들을 정의를 하고 병렬로 실행이 되어서 각각의 결과를 딕셔너리 형태로 반환한다.

In [40]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1}, 
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)
#한개의 단일 문서를 검색을 했을경우

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [42]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# RunnableParrellel 구성
runnable = RunnableParallel(
    {"context": itemgetter("context") , "question": itemgetter("question")}
)

"""
runnable = RunnableParallel(
    {"context:llm.invoke(itemgetter("context"))","questiom":chain.invoke(itmegetter("question"))}

)
외부의 입력값을 받고 내부에서 한가지의 방식이 아닌 여러가지 방식으로 처리를 할때 사용을 할 수 있다.
"""
# 객체를 실행
response = runnable.invoke({"context": retrieved_docs_text, "question": query})

# 응답을 출력
response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

`(2) RunnablePassthrough` -> 데이터 전달 역할을 보통한다.

In [45]:
from langchain_core.runnables import RunnablePassthrough
# 사용자의 입력값을 있는 그대로 전달을 한다. 여기서 RunnableParallel()메소드를 이용하기 때문에 딕셔너리 형태로 반환되어 나온다.
runnable = RunnableParallel(
    question=RunnablePassthrough(),
)

runnable.invoke({"query":"테슬라 창업자는 누구인가요?"})

{'question': {'query': '테슬라 창업자는 누구인가요?'}}

`(3) RunnableLambda` -> 데이터 변환을 하는데 보통 쓰인다.
- 정의: 파이썬의 커스텀 함수를 매핑하는데 사용

In [44]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())
# 값들을 RunnableParallel을 이용해서 한쪽은 RunnablePassthrough메소드에 처리를 하고 한쪽은 RunnableLambda을 이용해서 처리하는 체인으로 구성
runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words),
)

runnable.invoke("테슬라 창업자는 누구인가요?") 

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

### Runnable 인터페이스들을 잘 이해를 해야 파이프 라인을 분리하고 다시 합치고 등을 유연하게 반응을해서 시스템을 구현을 할 수 있다.

## 5. 전체 RAG 파이프라인 구성

`(1) RAG 프롬프트 템플릿`  

In [49]:
# Prompt 템플릿을 생성
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿을 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]



`(2) Retriever Chain 연결`  

In [50]:
# 벡터 검색기
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])
#하나의 단일 문자열로 결합
#리스트 속성으로 돌면서 작동

# 체인 구성
retriever_chain = retriever | format_docs

# 체인을 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


`(3) RAG Chain 연결`  

In [51]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=100)

# 체인 생성
# 기본적으로 RunnableParallel로 작동이 되고 있다
rag_chain = (
    {"context": retriever_chain , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

In [52]:
# 결과 출력
response

'마틴 에버하드와 마크 타페닝입니다.'

## 6. Gradio 챗봇

`(1) invoke 실행` 

In [55]:
#pip install gradio

In [59]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [60]:
# Graiio 종료
demo.close()

Closing server running on port: 7861


`(2) stream 실행` 

In [62]:
import gradio as gr
import time
def answer_invoke(message, history):
    partial_message = ""
    for chunk in rag_chain.stream(message):
        if chunk is not None:
            partial_message = partial_message + chunk
            time.sleep(0.1)
            yield partial_message

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gradio/components/chatbot.py:243: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [63]:
# Graiio 종료
demo.close()

Closing server running on port: 7862
